#### The code in this notebook performs several preprocessing tasks on the training dataset of Illinois basin oil well production data. The tasks include renaming columns, replacing missing and outlier values, filling missing dates, creating new features, and creating lagged features. The code also uses functions from the utils module to create additional features.

In [1]:
import pandas as pd
import numpy as np
import utils
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
df = pd.read_csv('./data/illinois_basing_train.csv')

# Rename target column
df = df.rename(columns={'inj_diff\xa0': 'Target', 'SampleTimeUTC': 'Date'})

# Replace Target outliers and missing values in two columns
df.at[836, 'Target'] = 15
df.at[837, 'Target'] = 30
df.at[838, 'Target'] = -44.5
df.at[839, 'Target'] = -0.5

df.loc[834, 'Avg_PLT_CO2VentRate_TPH'] = 0
df.loc[835, 'Avg_PLT_CO2VentRate_TPH'] = 0
df.loc[836, 'Avg_PLT_CO2VentRate_TPH'] = 0

# Fill empty 'hourly dates'
df = df.dropna(subset='Target')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df = df.set_index('Date').sort_index()
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
df = df.reindex(date_range, fill_value=np.nan)
df.reset_index(inplace=True, drop=False)
df = df.rename(columns={'index': 'Date'})

# Create a new column for the absolute value of the target variable
df['Abs Target'] = np.abs(df['Target'])

In [3]:
# Fill nan values from missing dates using linear regression
new_row_indices = df[df['Target'].isnull()].index
df = utils.impute_missing_dates(df, new_row_indices)

### Create classifier label (spikes)

In [4]:
df['Target > 2'] = df['Abs Target'].apply(lambda x: 1 if x > 2 else 0)

## Impute values

In [5]:
# Drop feature that has many null values in test set
for col in df.columns:
    if '6945' in col:
        df = df.drop(col, axis=1)

### Impute values manually for the most important feature

In [6]:
# Find indices of missing values
nan_indices = np.where(df['Avg_CCS1_WHCO2InjPs_psi'].isna())[0]

# Find consecutive sequences of missing values
nan_seq = []
temp_seq = []
for i, idx in enumerate(nan_indices):
    if i == 0:
        temp_seq.append(idx)
    elif idx == nan_indices[i-1]+1:
        temp_seq.append(idx)
    else:
        nan_seq.append(temp_seq)
        temp_seq = [idx]
# add the last sequence
if temp_seq:
    nan_seq.append(temp_seq)

# Replace missing values in specific locations with specific values
df.loc[nan_seq[0], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.203125
df.loc[nan_seq[1], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.203125
df.loc[nan_seq[2], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.203125
df.loc[nan_seq[3], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.203125
df.loc[nan_seq[4], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.233507
df.loc[nan_seq[5], 'Avg_CCS1_WHCO2InjPs_psi'] = 1.03
df.loc[nan_seq[6], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.2
df.loc[list(range(622,661)), 'Avg_CCS1_WHCO2InjPs_psi'] = 0.2
df.loc[nan_seq[9], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.2
df.loc[nan_seq[10], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.2
df.loc[nan_seq[11], 'Avg_CCS1_WHCO2InjPs_psi'] = 0.2

In [12]:
# Impute using K neighours found in the other notebook
imputer = KNNImputer(n_neighbors=4, weights='uniform')

# Split the data into predictor and target variables
y = df[['Date', 'Target', 'Abs Target', 'Target > 2']]
x = df.drop(['Date', 'Target', 'Abs Target', 'Target > 2'], axis=1)
cols = x.columns

# Fill missing values in the predictor variables
x = imputer.fit_transform(x)
x = pd.DataFrame(x, columns=cols)

In [9]:
# # Create a dictionary to store the missing value indices for each feature
# missing_value_indices = {}

# # Iterate over the columns in the DataFrame
# for column in df.columns:
#     # Find the indices of missing values in the current column
#     missing_indices = df[df[column].isnull()].index.tolist()
    
#     # If there are missing values, add the column name and indices to the dictionary
#     if missing_indices:
#         missing_value_indices[column] = missing_indices

# # Print the resulting dictionary
# print(missing_value_indices)

## Expand feature set

In [13]:
# Create temperature and pressure difference features
x = utils.get_temperature_diff_depth(x)
x = utils.get_pressure_diff_depth(x)

# Create a new feature by subtracting two existing features
x['Diff pressure downhole-wellhead'] = x['Avg_CCS1_DH6325Ps_psi'] - x['Avg_CCS1_ANPs_psi']

# Save column names and create lag features
cols = x.columns
lagged_df = utils.create_lag_features(x, features=cols, n_lags=5)

# Create additional features using functions from the utils module
x = utils.create_abs_features(x, x.columns)
x = utils.create_trend_features(x, x.columns)

c:\Users\Luis.Pinto1\Documents\GitHub\ML carbon competition\ML-carbon-capture-competition\utils.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Pressure diff 6416-4917 ft'] = df['Avg_VW1_Z07D6416Ps_psi'] - df['Avg_VW1_Z11D4917Ps_psi']
c:\Users\Luis.Pinto1\Documents\GitHub\ML carbon competition\ML-carbon-capture-competition\utils.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Pressure diff 5840-5653 ft'] = df['Avg_VW1_Z08D5840Ps_psi'] - df['Avg_VW1_Z09D5653Ps_psi']
c:\Users\Luis.Pinto1\Documents\GitHub\ML carb

In [14]:
# Replace infinite values with NaN
lagged_df = lagged_df.replace([np.inf, -np.inf], np.nan)

# Concatenate original DataFrame, lagged DataFrame, and target variables
df = pd.concat([x, lagged_df, y], axis=1)

# Create a new DataFrame with rounded values and drop the first five rows due to null lagged values
rounded_df = df.round(decimals=6)
rounded_df = rounded_df.drop(rounded_df.index[:5])

# Save the new DataFrame to a CSV file
rounded_df.to_csv('./data/train_df_expanded_KNN_imp_feats.csv', index=False)